初始化浏览器对象  
IDM命令
https://blog.csdn.net/MuoYangoRen/article/details/79954776

In [1]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# chrome 浏览器位置
# 驱动参数配置
options = Options()
# option.add_argument('headless')  # 后台运行
options.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"

# chrome 浏览器驱动位置
chrome_service = Service(r'C:\Users\root\anaconda3\Scripts\chromedriver.exe')


browser = webdriver.Chrome(
    options=options, 
    service =chrome_service)

参数配置

In [2]:
import logging
from selenium.webdriver.support.wait import WebDriverWait
logging.basicConfig(level=logging.INFO,
        format ='%(asctime)s-%(levelname)s: %(message)s')
index_url = "https://spa2.scrape.center/page/{page}"
time_out = 10 
total_page = 10 
wait= WebDriverWait(browser ,time_out)

In [3]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by  import By

def scrape_page(url,condition,locator):
    logging.info("scaping %s",url)
    try:
        browser.get(url)
        wait.until(condition(locator))
    except TimeoutException:
        logging.error("error occurred while scaping %s",url,exc_info=True)


def scrape_index(page):
    url = index_url.format(page=page)
    scrape_page(url,condition=EC.visibility_of_all_elements_located,locator=(By.CSS_SELECTOR,"#index .item"))
    


In [4]:
from urllib.parse import urljoin

def parse_index():

    # elements = browser.find_elements_by_css_selector('#index .item .name')
    elements = browser.find_elements(By.CSS_SELECTOR,'#index .item .name')
    for element in elements:
        href = element.get_attribute('href')
        yield urljoin(index_url, href)


def scrape_detail(url):
    scrape_page(url, condition=EC.visibility_of_element_located,
                locator=(By.TAG_NAME, 'h2'))

In [8]:
def parse_detail():
    url = browser.current_url
    name = browser.find_element(By.TAG_NAME,'h2').text
    categories = [element.text for element in browser.find_elements(By.CSS_SELECTOR,'.categories button span')]
    cover = browser.find_element(By.CSS_SELECTOR,'.cover').get_attribute('src')
    score = browser.find_element(By.CLASS_NAME,'score').text
    drama = browser.find_element(By.CSS_SELECTOR,'.drama p').text
    return {
        'url': url,
        'name': name,
        'categories': categories,
        'cover': cover,
        'score': score,
        'drama': drama
    }


In [6]:
import json
import os 
RESULTS_DIR = 'results'

os.path.exists(RESULTS_DIR) or os.makedirs(RESULTS_DIR)

def save_data(data):
    name = data.get('name')
    data_path = f'{RESULTS_DIR}/{name}.json'
    json.dump(data, open(data_path, 'w', encoding='utf-8'), ensure_ascii=False, indent=2)

In [ ]:
for page in range(1, total_page + 1):
    scrape_index(page)
    detail_urls = parse_index()
    for detail_url in list(detail_urls):
        logging.info('get detail url %s', detail_url)
        scrape_detail(detail_url)
        detail_data = parse_detail()
        logging.info('detail data %s', detail_data)
        save_data(detail_data)